# Projet spécialité Data-IA

# Analyse de données : l'industrie automobile et les changements climatiques

In [1]:
print("hello")

hello


Scrapper les données mondiales des ventes/émissions de CO2 des véhicules terrestre motorisés dans le monde, puis par continent, puis par pays de l'Europe, et pour une personne

# Scrapping des données mondiales

In [1]:
import requests # requêter sur le site
from bs4 import BeautifulSoup # parcourir le HTML
import pandas as pd # formatter les données/exporter en CSV

## Les ventes de véhicules, par années/pays/continent de 1999 à 2022

Nous avons trouvé le site de OICA. Une rubrique répertorie la production de véhicules motorisés dans le monde depuis 1999.

L'idée est donc de scraper ces données, dans la catégorie <b>production-statistics/{ANNÉE}-statistics</b>

Nous travaillerons depuis 2015

Problématique : le site sur lequel nous sommes tombés permet de récupérer les données en scrappant, ou en téléchargeant le format CSV. Les CSV permettent de connaitre les continents.
Mais la mise en page change souvent.

Pour les continents, un module Python existe, AWOC

# AWOC (test)

In [48]:
# Import the AWOC package.
import awoc

# Initialize the AWOC class.
my_world = awoc.AWOC()

# Let's retrieve the full list of nations of Europe.
countries_of_europe = my_world.get_countries_list_of('Europe')
print(countries_of_europe)
# It returns: [ 'Albania', 'Andorra', 'Austria', 'Belarus', 'Belgium', and so on... ]

# We can get the whole country data as well.
italy_data = my_world.get_country_data('Italy')

print(my_world.get_continents())

['Albania', 'Andorra', 'Austria', 'Belarus', 'Belgium', 'Bosnia and Herzegovina', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Faroe Islands', 'Finland', 'France', 'Germany', 'Gibraltar', 'Greece', 'Guernsey', 'Hungary', 'Iceland', 'Ireland', 'Isle of Man', 'Italy', 'Jersey', 'Kosovo', 'Latvia', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Macedonia', 'Malta', 'Moldova', 'Monaco', 'Montenegro', 'Netherlands', 'Norway', 'Poland', 'Portugal', 'Romania', 'Russia', 'San Marino', 'Serbia', 'Slovakia', 'Slovenia', 'Spain', 'Svalbard and Jan Mayen', 'Sweden', 'Switzerland', 'Ukraine', 'United Kingdom', 'Vatican']
[{'Continent Code': 'af', 'Continent Name': 'Africa'}, {'Continent Code': 'an', 'Continent Name': 'Antarctica'}, {'Continent Code': 'as', 'Continent Name': 'Asia'}, {'Continent Code': 'eu', 'Continent Name': 'Europe'}, {'Continent Code': 'na', 'Continent Name': 'North America'}, {'Continent Code': 'oc', 'Continent Name': 'Oceania'}, {'Continent Code': 'sa

In [ ]:
import pycountry_convert as pc

def get_continent(country_alpha2):
    try:
        continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        continent_name = pc.convert_continent_code_to_continent_name(continent_code)
        return continent_name
    except ValueError:
        return None

# Exemple d'utilisation
country_alpha2 = "US"  # Code ISO 3166-1 alpha-2 des États-Unis
continent = get_continent(country_alpha2)

if continent:
    print(f"Le continent de {country_alpha2} est {continent}.")
else:
    print(f"Impossible de déterminer le continent pour {country_alpha2}.")


In [9]:
# CTRL + '/' pour commenter plusieurs lignes à la fois

import datetime

def get_current_year() -> int: 
    """ 
    for automatisation of the dataset. In 2025, 
    the program will research for the 2025's web page
    """
    today = datetime.date.today()
    year = today.year
    print(year)

    return year

def scrape_page(soup: BeautifulSoup, world_production_list: list, start, end):
    table = soup.find("tbody")
    countries = table.find_all("tr")

    # print(countries)

    for country_tr in countries:
        country_name = country_tr.find("td", {"class": f"column-1"}).text
        nbr_cars = country_tr.find("td", {"class": f"column-2"}).text
        nbr_commercial_vehicle = country_tr.find("td", {"class": f"column-3"}).text
        total = country_tr.find("td", {"class": f"column-4"}).text
        percent_change = country_tr.find("td", {"class": f"column-5"}).text

        # print(country_tr)
        # print(country_name)

        dict = {
            'Year': '',
            'Country_Name': '',
            'Nbr_Cars': '',
            'Nbr_Commercial_Vehicle': '',
            'Total': '',
            'Percent_Change': ''
        }

        dict['Year'] = start
        dict["Country_Name"] = country_name
        dict["Nbr_Cars"] = nbr_cars
        dict["Nbr_Commercial_Vehicle"] = nbr_commercial_vehicle
        dict["Total"] = total
        dict["Percent_Change"] = percent_change

        world_production_list.append(dict)

    start += 1

    if start < end_year:
        print('Accès à la page suivante')
        next_url = f'{target}{start}-statistics'
        print(f"Next URL : {next_url}")
        req = requests.get(target +  next_url)
        next_soup = BeautifulSoup(req.text, "html.parser")
        scrape_page(next_soup, world_production_list, start, end)

    return 

begin_year = 2015
end_year = get_current_year() - 1

target = f'https://www.oica.net/category/production-statistics/'
# https://www.oica.net/category/production-statistics/2015-statistics/
req = requests.get(target + f'{begin_year}-statistics')

if req.status_code != 200:
    print("Erreur")
    exit(0)
else:
    print(f"Accès au site {target}, status = {req.status_code}")

soup = BeautifulSoup(req.text, "html.parser")

if __name__ == "__main__":
    print(f"Date de début {begin_year}; Date de fin : {end_year}")

    world_production_list = []
    # print(world_production_list)
    scrape_page(soup, world_production_list, begin_year, end_year)
    print(world_production_list)
    df = pd.DataFrame(world_production_list)
    print(df.head(10))
    # df.insert(0, 'ID', range(1, len(quote_list) +1))
    df.to_csv(f'word_car_production_{begin_year}_{end_year}.csv', index=False, sep=';')


2024
Accès au site https://www.oica.net/category/production-statistics/, status = 200
Date de début 2015; Date de fin : 2023
Accès à la page suivante
Next URL : https://www.oica.net/category/production-statistics/2016-statistics
Accès à la page suivante
Next URL : https://www.oica.net/category/production-statistics/2017-statistics
Accès à la page suivante
Next URL : https://www.oica.net/category/production-statistics/2018-statistics
Accès à la page suivante
Next URL : https://www.oica.net/category/production-statistics/2019-statistics
Accès à la page suivante
Next URL : https://www.oica.net/category/production-statistics/2020-statistics
Accès à la page suivante
Next URL : https://www.oica.net/category/production-statistics/2021-statistics
Accès à la page suivante
Next URL : https://www.oica.net/category/production-statistics/2022-statistics
[{'Year': 2015, 'Country_Name': 'Argentina', 'Nbr_Cars': '308,756', 'Nbr_Commercial_Vehicle': '224,927', 'Total': '533,683', 'Percent_Change': '-13

# Amélioration du dataset

## Rajout des continents

Afin de réaliser notre storytelling, nous souhaitons faire une étude par pays, mais également pas continent et dans le monde. Il faut alors ajouter une colonne continent pour nos données

In [15]:
import pandas as pd
import pycountry_convert as pc

In [16]:
df = pd.read_csv("./word_car_production_2015_2023.csv", delimiter=";")

df.head()

,Year,Country_Name,Nbr_Cars,Nbr_Commercial_Vehicle,Total,Percent_Change
0,2015,Argentina,"308,756","224,927","533,683",-13.5%
1,2015,Australia,"159,872","13,137","173,009",-4.0%
2,2015,Austria,"109,000","16,500","125,500",-17.4%
3,2015,Belgium,"369,172","40,168","409,340",-20.8%
4,2015,Brazil,"2,018,954","410,509","2,429,463",-22.8%


In [11]:
df.columns

Index(['Year', 'Country_Name', 'Nbr_Cars', 'Nbr_Commercial_Vehicle', 'Total',
       'Percent_Change'],
      dtype='object')

Corriger les erreurs de noms de pays

In [63]:
# Correct format for Czech Republic
df['Country_Name'] = df['Country_Name'].replace('Czech Rep.', 'Czech Republic')

# Format correct for United Kingdom
df['Country_Name'] = df['Country_Name'].replace('UK', 'United Kingdom')

In [64]:
df.head(10)

,Year,Country_Name,Nbr_Cars,Nbr_Commercial_Vehicle,Total,Percent_Change,Continent
0,2015,Argentina,"308,756","224,927","533,683",-13.5%,South America
1,2015,Australia,"159,872","13,137","173,009",-4.0%,Oceania
2,2015,Austria,"109,000","16,500","125,500",-17.4%,Europe
3,2015,Belgium,"369,172","40,168","409,340",-20.8%,Europe
4,2015,Brazil,"2,018,954","410,509","2,429,463",-22.8%,South America
5,2015,Canada,"888,565","1,394,909","2,283,474",-4.6%,North America
6,2015,China,"21,079,427","3,423,899","24,503,326",3.3%,Asia
7,2015,Czech Republic,"1,298,236","5,367","1,303,603",4.2%,Europe
8,2015,Egypt,"12,000","24,000","36,000",-15.3%,Africa
9,2015,Finland,"69,000",53,"69,053",53.3%,Europe


In [70]:
def get_continent_code(row):
    try:
        if row.Country_Name.lower() == 'total':
            return 'Entire World'
        
        if row.Country_Name.lower() == 'others':
            return 'Other'
            
        country_alpha2 = pc.country_name_to_country_alpha2(row.Country_Name.title() if row.Country_Name != 'USA' else row.Country_Name, cn_name_format="default")
        # print(row.Country_Name)
        continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        continent_name = pc.convert_continent_code_to_continent_name(continent_code)
        
        return continent_name
    
    except (ValueError, KeyError, TypeError) as e:
        print(f"Erreur lors de la conversion pour {row.Country_Name}: {e}")
        return None

In [71]:
df['Continent'] = df.apply(get_continent_code, axis=1)

In [72]:
df.head(41)

,Year,Country_Name,Nbr_Cars,Nbr_Commercial_Vehicle,Total,Percent_Change,Continent
0,2015,Argentina,"308,756","224,927","533,683",-13.5%,South America
1,2015,Australia,"159,872","13,137","173,009",-4.0%,Oceania
2,2015,Austria,"109,000","16,500","125,500",-17.4%,Europe
3,2015,Belgium,"369,172","40,168","409,340",-20.8%,Europe
4,2015,Brazil,"2,018,954","410,509","2,429,463",-22.8%,South America
5,2015,Canada,"888,565","1,394,909","2,283,474",-4.6%,North America
6,2015,China,"21,079,427","3,423,899","24,503,326",3.3%,Asia
7,2015,Czech Republic,"1,298,236","5,367","1,303,603",4.2%,Europe
8,2015,Egypt,"12,000","24,000","36,000",-15.3%,Africa
9,2015,Finland,"69,000",53,"69,053",53.3%,Europe


In [20]:
import pycountry_convert as pc

def get_continent(country_name):
    try:
        # Convertir le nom du pays en code ISO 3166-1 alpha-2
        country_alpha2 = pc.country_name_to_country_alpha2(country_name, cn_name_format="default")
        
        # Obtenir le continent à partir du code alpha-2
        continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        continent_name = pc.convert_continent_code_to_continent_name(continent_code)
        
        return continent_name
    except (ValueError, KeyError):
        return None

# Exemple d'utilisation pour le Brésil
country_name = "Czech Republic"
country_name = country_name.title()
print(country_name.title())
continent = get_continent(country_name)

if continent:
    print(f"Le continent de {country_name} est {continent}.")
else:
    print(f"Impossible de déterminer le continent pour {country_name}.")


Czech Republic
Le continent de Czech Republic est Europe.


c:\Python310\lib\site-packages\pycountry\db.py:51: UserWarning: Country's official_name not found. Country name provided instead.
  warnings.warn(warning_message, UserWarning)
c:\Python310\lib\site-packages\pycountry\db.py:51: UserWarning: Country's common_name not found. Country name provided instead.
  warnings.warn(warning_message, UserWarning)
